# 피부질환 Classification 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
from torchvision import transforms
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
class CustomImageDataset(Dataset):
    def read_data_set(self):

        all_img_files = []
        all_labels = []

        class_names = os.walk(self.data_set_path).__next__()[1]

        for index, class_name in enumerate(class_names):
            label = index
            img_dir = os.path.join(self.data_set_path, class_name)
            img_files = os.walk(img_dir).__next__()[2]

            for img_file in img_files:
                img_file = os.path.join(img_dir, img_file)
                img = Image.open(img_file)
                if img is not None:
                    all_img_files.append(img_file)
                    all_labels.append(label)

        return all_img_files, all_labels, len(all_img_files), len(class_names)

    def __init__(self, data_set_path, transforms=None):
        self.data_set_path = data_set_path
        self.image_files_path, self.labels, self.length, self.num_classes = self.read_data_set()
        self.transforms = transforms

    def __getitem__(self, index):
        image = Image.open(self.image_files_path[index])
        image = image.convert("RGB")

        if self.transforms is not None:
            image = self.transforms(image)

        return {'image': image, 'label': self.labels[index]}

    def __len__(self):
        return self.length

In [ ]:
# # 드롭아웃 적용 O

# class Convbox(nn.Module):
#     def __init__(self, in_channels, out_channels, stride=1, dropout_rate=0.1):
#         super(Convbox, self).__init__()
#         # 첫 번째 컨볼루션 레이어
#         self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn1 = nn.BatchNorm2d(out_channels)
#         self.relu = nn.ReLU(inplace=True)
#         self.dropout = nn.Dropout2d(p=dropout_rate)
#         # 두 번째 컨볼루션 레이어
#         self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
#         self.bn2 = nn.BatchNorm2d(out_channels)

#         #  네트워크의 깊이를 늘릴 때 발생하는 그래디언트 소실 문제를 완화(앞선의 입력을 더해줌으로써)
#         self.storage = nn.Sequential()
#         if stride != 1 or in_channels != out_channels:
#             self.storage = nn.Sequential(
#                 nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
#                 nn.BatchNorm2d(out_channels)
#             )

#     def forward(self, x):
#         storage = self.storage(x)  # 스킵 연결 부분을 먼저 계산
#         out = self.conv1(x)
#         out = self.bn1(out)
#         out = self.relu(out)
#         out = self.dropout(out)  # dropout층 추
#         out = self.conv2(out)
#         out = self.bn2(out)
#         if storage.size(2) != out.size(2) or storage.size(3) != out.size(3):
#             storage = F.interpolate(storage, size=(out.size(2), out.size(3)), mode='nearest')
#         out += storage  # 스킵 연결을 더함
#         out = self.relu(out)
#         return out

# class CustomSkinClassification(nn.Module):
#     def __init__(self, num_classes,dropout_rate=0.1):
#         super(CustomSkinClassification, self).__init__()

#         # 초기 컨볼루션 레이어
#         self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
#         self.bn1 = nn.BatchNorm2d(64)
#         self.relu = nn.ReLU(inplace=True)
#         self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
#         self.dropout = nn.Dropout2d(p=dropout_rate)


#         # Convbox Layer 1
#         self.layer1 = nn.Sequential(
#             Convbox(64, 64),
#             Convbox(64, 64),
#             Convbox(64, 64)
#         )

#         # Convbox Layer 2
#         self.layer2 = nn.Sequential(
#             Convbox(64, 128, stride=2),
#             Convbox(128, 128),
#             Convbox(128, 128),
#             Convbox(128, 128, dropout_rate=dropout_rate)
#         )

#         # Convbox Layer 3
#         self.layer3 = nn.Sequential(
#             Convbox(128, 256, stride=2),
#             Convbox(256, 256),
#             Convbox(256, 256),
#             Convbox(256, 256),
#             Convbox(256, 256),
#             Convbox(256, 256,dropout_rate=dropout_rate)
#         )

#         # Convbox Layer 4
#         self.layer4 = nn.Sequential(
#             Convbox(256, 512, stride=2),
#             Convbox(512, 512),
#             Convbox(512, 512,dropout_rate=dropout_rate)
#         )

#         # Global Average Pooling 및 Fully Connected Layer
#         self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
#         self.fc = nn.Linear(512, num_classes)

#     def forward(self, x):
#         x = self.conv1(x)
#         x = self.bn1(x)
#         x = self.relu(x)
#         x = self.maxpool(x)

#         x = self.layer1(x)
#         x = self.layer2(x)
#         x = self.layer3(x)
#         x = self.layer4(x)

#         x = self.avgpool(x)
#         x = torch.flatten(x, 1)
#         x = self.fc(x)

#         return x

In [ ]:
# 드롭아웃 적용 X

class Convbox(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(Convbox, self).__init__()
        # 첫 번째 컨볼루션 레이어
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        # 두 번째 컨볼루션 레이어
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        #  네트워크의 깊이를 늘릴 때 발생하는 그래디언트 소실 문제를 완화(앞선의 입력을 더해줌으로써)
        self.storage = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.storage = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        storage = self.storage(x)  # 스킵 연결 부분을 먼저 계산
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if storage.size(2) != out.size(2) or storage.size(3) != out.size(3):
            storage = F.interpolate(storage, size=(out.size(2), out.size(3)), mode='nearest')
        out += storage  # 스킵 연결을 더함
        out = self.relu(out)
        return out

class CustomSkinClassification(nn.Module):
    def __init__(self, num_classes):
        super(CustomSkinClassification, self).__init__()

        # 초기 컨볼루션 레이어
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Convbox Layer 1
        self.layer1 = nn.Sequential(
            Convbox(64, 64),
            Convbox(64, 64),
            Convbox(64, 64)
        )

        # Convbox Layer 2
        self.layer2 = nn.Sequential(
            Convbox(64, 128, stride=2),
            Convbox(128, 128),
            Convbox(128, 128),
            Convbox(128, 128)
        )

        # Convbox Layer 3
        self.layer3 = nn.Sequential(
            Convbox(128, 256, stride=2),
            Convbox(256, 256),
            Convbox(256, 256),
            Convbox(256, 256),
            Convbox(256, 256),
            Convbox(256, 256)
        )

        # Convbox Layer 4
        self.layer4 = nn.Sequential(
            Convbox(256, 512, stride=2),
            Convbox(512, 512),
            Convbox(512, 512)
        )

        # Global Average Pooling 및 Fully Connected Layer
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)

        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)

        return x

In [ ]:
# 출력 tensor가 맞는지 확인

from torchsummary import summary
summary(CustomSkinClassification(num_classes=4), input_size=(3, 224, 224))

In [ ]:
hyper_param_epoch = 10
hyper_param_batch = 64
hyper_param_learning_rate = 0.001

resize_train_mean=[0.485, 0.456, 0.406]
resize_train_std=[0.229, 0.224, 0.225]

resize_test_mean=[0.485, 0.456, 0.406]
resize_test_std=[0.229, 0.224, 0.225]

# # 둘 중 하나 택1 (위에 것하고)
# resize_train_mean=[0.17191947, 0.41128376, 0.56153077]
# resize_train_std=[0.16150557, 0.16577946, 0.16063999]

# resize_test_mean=[0.15918699, 0.410329, 0.55247366]
# resize_test_std=[0.1542138, 0.16098696, 0.15552239]


# 선택.1
transform_train = transforms.Compose([
    transforms.Resize((256, 256)), # 이미지 resize
    transforms.RandomCrop(224), # 이미지를 랜덤으로 크롭
    transforms.ToTensor(),
    transforms.Normalize(resize_train_mean, resize_train_std)
])

transform_test = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.CenterCrop(224), # 이미지를 랜덤으로 크롭
    transforms.ToTensor(),
    transforms.Normalize(resize_test_mean, resize_test_std)
])


# 선택.2
# transform_train = transforms.Compose([
#     transforms.Resize((128, 128)), # 이미지 resize
#     transforms.RandomCrop(124), # 이미지를 랜덤으로 크롭
#     transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2), # 이미지 지터링(밝기, 대조, 채비, 색조)
#     transforms.RandomHorizontalFlip(p = 1), # p확률로 이미지 좌우반전
#     transforms.RandomVerticalFlip(p = 1), # p확률로 상하반전
#     transforms.ToTensor(),
#     transforms.Normalize(resize_train_mean, resize_train_std)
# ])

# transform_test = transforms.Compose([
#     transforms.Resize((128, 128)), 
#     transforms.ToTensor(),
#     transforms.Normalize(resize_test_mean, resize_test_std)
# ])


# # 선택.3
# transform_train = transforms.Compose([
#     transforms.Resize((256, 256)), # 이미지 resize
#     transforms.ToTensor()
# ])

# transform_test = transforms.Compose([
#     transforms.Resize((256, 256)),
#     transforms.ToTensor()
# ])

train_data_set = CustomImageDataset(data_set_path="/content/drive/MyDrive/data/version1/train", transforms=transforms_train)
train_loader = DataLoader(train_data_set, batch_size=hyper_param_batch, shuffle=True)

test_data_set = CustomImageDataset(data_set_path="/content/drive/MyDrive/data/version1/test", transforms=transforms_test)
test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=False)

In [ ]:
# # 드롭아웃 적용 O
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# num_classes = train_data_set.num_classes
# custom_model = CustomSkinClassification(num_classes=num_classes,dropout_rate=0.1).to(device)

In [ ]:
# 드롭아웃 적용 X
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

num_classes = train_data_set.num_classes
custom_model = CustomSkinClassification(num_classes=num_classes).to(device)

In [ ]:
# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(custom_model.parameters(), lr=hyper_param_learning_rate)

In [ ]:
for e in range(hyper_param_epoch):
    custom_model.train()
    correct_train = 0
    total_train = 0

    for i_batch, item in enumerate(train_loader):
        images = item['image'].to(device)
        labels = item['label'].to(device)

        # Forward pass
        outputs = custom_model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        _, predicted_train = torch.max(outputs.data, 1)
        total_train += labels.size(0)
        correct_train += (predicted_train == labels).sum().item()

        if (i_batch + 1) % hyper_param_batch == 0:
            print('에포크 [{}/{}], 배치 [{}/{}], 훈련 손실: {:.4f}'
                  .format(e + 1, hyper_param_epoch, i_batch + 1, len(train_loader), loss.item()))

    # 훈련 정확도 기록
    train_accuracy = 100 * correct_train / total_train
    train_accuracies.append(train_accuracy)
    print('훈련데이터 {}개에 대한 정확도: {} %'.format(total_train, train_accuracy))

    # Test the model
    custom_model.eval()
    with torch.no_grad():
        correct_test = 0
        total_test = 0

        for item in test_loader:
            images = item['image'].to(device)
            labels = item['label'].to(device)
            outputs = custom_model(images)
            _, predicted_test = torch.max(outputs.data, 1)
            total_test += len(labels)
            correct_test += (predicted_test == labels).sum().item()

        # 테스트 정확도 기록
        test_accuracy = 100 * correct_test / total_test
        test_accuracies.append(test_accuracy)
        print('테스트 데이터 {}개에 대한 정확도: {} %'.format(total_test, test_accuracy))


In [ ]:
# 정확도 시각화

plt.plot(train_accuracies, label='Train Accuracy')
plt.plot(test_accuracies, label='Test Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy (%)')
plt.legend()
plt.show()


In [ ]:
# 결과 시각화
label_tags = {0: 'acne', 1: 'Eczema', 2: 'normal_face_skin', 3: 'skin_cancer'}

custom_model.eval()

fig, axes = plt.subplots(4, 4, figsize=(10, 10))
fig.suptitle('Skin Condition Classification Predictions', fontsize=16)

plt.subplots_adjust(wspace=0.5, hspace=0.5)

with torch.no_grad():
    
    test_loader = DataLoader(test_data_set, batch_size=hyper_param_batch, shuffle=True)
    data_iter = iter(test_loader)

    for i in range(4):
        for j in range(4):
            data = next(data_iter)

            images = data['image'].to(device)
            labels = data['label'].to(device)

            outputs = custom_model(images)
            _, predicted_classes = torch.max(outputs.data, 1)
            predicted_labels = [label_tags[pred.item()] for pred in predicted_classes]

            resized_image = transforms.functional.resize(images[0], (128, 128))

            title = f'Actual - {label_tags[labels[0].item()]},\n Predicted - {predicted_labels[0]}'

            correct_prediction = (predicted_classes[0] == labels[0]).item()

            axes[i, j].imshow(resized_image.permute(1, 2, 0).cpu().numpy())
            axes[i, j].set_title(title, fontsize=8)
            axes[i, j].axis('off')

            annotation_text = 'Success' if correct_prediction else 'Failure'
            annotation_color = 'green' if correct_prediction else 'red'
            axes[i, j].annotate(annotation_text,
                                xy=(0.5, 0.02), xycoords='axes fraction',
                                ha='center', va='center', color=annotation_color,
                                bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

plt.show()

In [ ]:
#시현코드

import torch
import torch.nn.functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
from PIL import Image
import matplotlib.pyplot as plt

torch.save(custom_model.state_dict(), 'path_to_save_model.pth')
custom_model.load_state_dict(torch.load('path_to_save_model.pth'))

custom_model.eval()


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
custom_model.to(device)


transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])


image_path = '/content/drive/MyDrive/data/시현2.png'
image = Image.open(image_path).convert('RGB')


image = transform(image).unsqueeze(0).to(device)


with torch.no_grad():
    output = custom_model(image)
    probabilities = F.softmax(output, dim=1)
    _, predicted_class = torch.max(output.data, 1)
    predicted_label = label_tags[predicted_class.item()]
    predicted_probability = probabilities[0, predicted_class[0]].item()


resized_image = transforms.functional.resize(image[0], (128, 128))

plt.imshow(resized_image.permute(1, 2, 0).cpu().numpy())
plt.title(f'Predicted Class: {predicted_label}, Probability: {predicted_probability:.2f}')
plt.axis('off')
plt.show()
